This code produces all figures as well as summary statistics reported in the main paper and supplement. It has to be run after the analysis code `OIC-OO v7` and the results processing code `OSM Results Processing`, using the same ControlFile and from the same folder; this code will read the processed .tab files from the analysis, and will fail if they are not available. See main `Analysis & Graphing` folder for `ReadMe` with explanation of ControlFile format and fields.

In [ ]:
import os
import json
import regex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
import matplotlib.cm as cm

from shutil import copy
from distutils.dir_util import copy_tree


def get_value(file, varname):
    """General purpose function for reading values from .mdl, .out, etc. 
    files; returns value matching `varname` in a 'var = val' syntax"""
    varregex = regex.compile(r'(?<=([^\w ]|\n)\s?' + regex.escape(varname)
                             + r'\s*=)\s*-?(?:\d*)(\.\d*)?([eE][+\-]?\d+)?')

    with open(file, 'r') as f:
        filetext = f.read()
        value = float((regex.search(varregex, filetext))[0])

    return value


def strbds_from_perc(perc):
    """Return lower & upper bounds that define `perc` CrI as strings"""
    if perc > 1: # If perc specified as percentage (not decimal)
        return [str(round((0.5 - perc/200), 3)), str(round((0.5 + perc/200), 3))]
    else: # If perc specified as decimal, not 100%
        return [str(round((0.5 - perc/2), 3)), str(round((0.5 + perc/2), 3))]

In [ ]:
# Read specified controlfile and unpack into variables
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

for k,v in cf.items():
    exec(k + '=v')

for setting in [analysissettings]:
    for k, v in setting.items():
        exec(k + '=v')

# Initialise base working directory
os.chdir(f"{baserunname}_IterCal")
basedir = os.getcwd()
os.chdir('./Results')

In [ ]:
# Establish dictionary of variables and labels, adding entries as needed
fitdict = dict(fitlist)
fitdict['ROUT'] = 'Total Rx opioid use disorder'
fitdict['InRT'] = 'Total Rx misuse initiation'
fitdict['ODRT'] = 'Overdose deaths (Rx + Rx synthetics)'
    
# Define y-axis labels for main variable plots
fits_y_labels = ['Incidence (millions/year)', 
                 'Use disorder prevalence (millions)', 
                 'Overdose deaths (thousands/year)']

# Specify all vars to be included
fullfits = ['InRM', 'InRD', 'RMis', 'NDHU', 'ROUD', 'ROUH', 'HUD', 'TxBp', 
            'InHD', 'InHR', 'InHO', 'InHT', 'ODRB', 'ODSB', 'ODHC', 'ODTo']

In [ ]:
##### PLOT MAIN FITS AND PROJECTIONS #####

def plot_fits_main(fits, mainfits, fitdict, ylabels, errmult=1.96):
    """Plot fit-to-data for six variables specified in `mainfits` list, 
    with names and y-labels from `fitdict` and `ylabels`, and error bars 
    where available of `errmult` x stderr"""
    
    # Define main figure and axes
    fig0, ((ax0, ax1), (ax2, ax3), (ax4, ax5)) = plt.subplots(
        3, 2, figsize=[7, 7], sharex=True, constrained_layout=True)
    ax_list = [ax0, ax1, ax2, ax3, ax4, ax5]

    # Plot sim output, data, and error bars where available
    for ax, var in zip(ax_list, mainfits):
        ax.plot(fits.columns, fits.loc[f'SimVar[{var}]'], zorder=10, label='Simulated')
        ax.scatter(fits.columns[::4], fits.loc[f'DataVar[{var}]'][::4], 
                   color='silver', label='Historical')
        try:
            # Compile errorbars array limiting lower error bounds to just above zero
            lowers = np.minimum(fits.loc[f'DataVar[{var}]'][::4] / (errmult + 0.01), 
                                fits.loc[f'DataErr[{var}]'][::4]) * errmult
            uppers = fits.loc[f'DataErr[{var}]'][::4] * errmult # Use errmult=1.96 for 95%CI
            errors = np.array(list(zip(lowers, uppers))).T # Concatenate and transpose array
            ax.errorbar(fits.columns[::4], fits.loc[f'DataVar[{var}]'][::4], yerr=errors, 
                        fmt='o', color='silver', lw=1, capsize=3, label='Historical')
        except KeyError: # Catch exception when DataErr not available
            pass
        ax.set_title(fitdict[var], fontsize=8)
        
        # Anchor y axis on 0 and scale display values
        ax.set_ylim(0)
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e6:.1f}')
        ax.tick_params(axis='y', labelsize=6)

    # Format X and Y axis for bottom two axes
    for ax in [ax4, ax5]:
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=6, labelrotation=90)
        
        # Override Y-axis scaling
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e3:.0f}')

    # Set labels for left-hand axes only, and figure title
    for ax, label in zip([ax0, ax2, ax4], ylabels):
        ax.set_ylabel(label, fontsize=8)
    fig0.suptitle('Model fit to data for selected variables', fontsize=10)
    
    return fig0

    
def plot_fits_proj(table, senstable, projfits, fitdict, bounds, ylabels, 
                   basevar='SimVar', endyear='2020'):
    """Plot projections for variables specified in `projfits` list, with 
    CIs specified in `bounds` (multiple CIs allowed) and names and y-
    labels from `fitdict` and `ylabels`; uses SimVar by default but can 
    set `basevar` to RepVar instead to show measurement noise"""
    t, s = table, senstable # For more compact code
    
    # Define main figure and axes
    fig0, ((ax0, ax1), (ax2, ax3), (ax4, ax5)) = plt.subplots(
        3, 2, figsize=[7, 7], sharex=True, constrained_layout=True)
    ax_list = [ax0, ax1, ax2, ax3, ax4, ax5]

    # Plot sim output and CI bounds
    for ax, var in zip(ax_list, projfits):
        ax.plot(t.columns, t.loc[f'{basevar}[{var}]'])
        for bd in bounds:
            ax.fill_between(t.columns[::4], s.loc[f'{basevar}[{var}]', bd[0]][::4], 
                            s.loc[f'{basevar}[{var}]', bd[1]][::4], alpha=0.2)
        ax.set_title(fitdict[var], fontsize=8)
        
        # Add vertical dashed line to mark end of data
        ax.vlines(endyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
        
        # Anchor y axis on 0 and scale display values
        ax.set_ylim(0)
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e6:.1f}')
        ax.tick_params(axis='y', labelsize=6)

    # Format X and Y axis for bottom two axes
    for ax in [ax4, ax5]:
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=6, labelrotation=90)
        
    # Set shared Y-axes for left and right panels
    ax1.sharey(ax0)
    ax3.sharey(ax2)
    ax4.sharey(ax5)
    
    # Override Y-axis scaling
    ax4.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e3:.1f}')
        
    # Set labels for left-hand axes only, and figure title
    for ax, label in zip([ax0, ax2, ax4], ylabels):
        ax.set_ylabel(label, fontsize=8)
    fig0.suptitle('Model base case projections for selected model outcomes', fontsize=10)
    
    # Create figure legend with proxy artists for labels
    etc = mlines.Line2D([], [], label='Exogenous Trends Continue (ETC)', color='tab:blue')
    hist = mlines.Line2D([], [], label='Projection start time', color='r', ls='dashed')
    handles = [etc, hist]
    fig0.legend(handles=handles, loc=9, bbox_to_anchor=(0.5, 0), ncol=2, fontsize=8)
    
    return fig0


# Establish dictionary of variables and labels, adding entries as needed
fitdict = dict(fitlist)
fitdict['ROUT'] = 'Total Rx opioid use disorder'
fitdict['InRT'] = 'Total Rx misuse initiation'
fitdict['ODRT'] = 'Overdose deaths (Rx + Rx synthetics)'
    
# Define y-axis labels for main variable plots
fits_y_labels = ['Incidence (millions/year)', 
                 'Use disorder prevalence (millions)', 
                 'Overdose deaths (thousands/year)']

# Create basic fit-to-data graph for selected variables
fits = pd.read_csv(f'{baserunname}_main_fits.tab', sep='\t', index_col=0)
fig0 = plot_fits_main(fits, mainfits, fitdict, fits_y_labels)

# Create basic projections graph for selected variables
# t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', 
#                 sep='\t', index_col=0).drop_duplicates() # Ignore duplicate summed rows
# s = pd.read_csv(f'{baserunname}_sens_{basescens[0][:-4]}_clean.tab', 
#                 sep='\t', index_col=[0,1]).drop_duplicates() # Ignore duplicate summed rows
t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
s = pd.read_csv(f'{baserunname}_sens_{basescens[0][:-4]}_clean.tab', sep='\t', index_col=[0,1])
fig1 = plot_fits_proj(t, s, projfits, fitdict, bounds, fits_y_labels, endyear=str(endyear))
fig2 = plot_fits_proj(t, s, projfits, fitdict, bounds, fits_y_labels, endyear=str(endyear), basevar='RepVar')

fig0.savefig(f"./{baserunname}_fits_main.{figext}", dpi=figdpi, bbox_inches='tight')
fig1.savefig(f"./{baserunname}_fits_proj.{figext}", dpi=figdpi, bbox_inches='tight')
fig2.savefig(f"./{baserunname}_fits_proj_noise.{figext}", dpi=figdpi, bbox_inches='tight')

In [ ]:
### PLOT TRIPLE SCENARIO PROJECTIONS FIGURE ###

def plot_fits_proj_scens(baserunname, scens, projfits, fitdict, bounds, 
                         ylabels, basevar='SimVar', endyear='2020'):
    """Plot projections across multiple scenarios from `scens` for six 
    vars specified in `projfits` list, with CrIs specified in `bounds` 
    (multiple CrIs allowed, use paired lower/upper values) and names and 
    y-labels from `fitdict` and `ylabels`; uses SimVar by default but 
    can set `basevar` to RepVar instead to show measurement noise"""
    
    # Define main figure and axes
    fig0, ((ax0, ax1), (ax2, ax3), (ax4, ax5)) = plt.subplots(
        3, 2, figsize=[7, 7], sharex=True, constrained_layout=True)
    ax_list = [ax0, ax1, ax2, ax3, ax4, ax5]
        
    # Plot sim output and CI bounds for each scenario
    for ax, var in zip(ax_list, projfits):
        for i, scen in enumerate(scens):
            # Read in main and sensitivity output files
            t = pd.read_csv(f'{baserunname}_final_{scen[:-4]}_vars.tab', sep='\t', index_col=0)
            s = pd.read_csv(f'{baserunname}_sens_{scen[:-4]}_clean.tab', sep='\t', index_col=[0,1])

            if i > 0: # Subset to just projection portion for all but first scenario
                t = t.loc[:, endyear:]
                s = s.loc[:, endyear:]
                
            # Plot lines with each consecutive scenario lower in zorder
            ax.plot(t.columns, t.loc[f'{basevar}[{var}]'], zorder=(3-i/10))
            for bd in bounds:
                ax.fill_between(t.columns[::4], s.loc[f'{basevar}[{var}]', bd[0]][::4], 
                                s.loc[f'{basevar}[{var}]', bd[1]][::4], alpha=0.2, zorder=(3-i/10))

        ax.set_title(fitdict[var], fontsize=8) # Set axes titles based on fitdict
        
        # Add vertical dashed line to mark end of data
        ax.vlines(endyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
        
        # Anchor y axis on 0 and scale display values
        ax.set_ylim(0)
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e6:.1f}')
        ax.tick_params(axis='y', labelsize=6)

    # Format X and Y axis for bottom two axes
    for ax in [ax4, ax5]:
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=6, labelrotation=90)
        
    # Set shared Y-axes for left and right panels
    ax1.sharey(ax0)
    ax3.sharey(ax2)
    ax4.sharey(ax5)
    
    # Override Y-axis scaling
    ax4.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e3:.1f}')
        
    # Set labels for left-hand axes only, and figure title
    for ax, label in zip([ax0, ax2, ax4], ylabels):
        ax.set_ylabel(label, fontsize=8)
    fig0.suptitle('Model base case projections for selected model outcomes', fontsize=10)
    
    # Create figure legend with proxy artists for labels
    base = mlines.Line2D([], [], label='Exogenous Trends Continue (ETC)', color='tab:blue')
    pos = mlines.Line2D([], [], label='Optimistic Scenario', color='tab:orange')
    neg = mlines.Line2D([], [], label='Pessimistic Scenario', color='tab:green')
    hist = mlines.Line2D([], [], label='Projection start time', color='r', ls='dashed')
    handles = [base, pos, neg, hist]
    fig0.legend(handles=handles, loc=9, bbox_to_anchor=(0.5, 0), ncol=4, fontsize=8)
        
    return fig0


# Create basic projections graph for selected variables
scens = basescens[0:1] + scenariolist[0:2]
fig0 = plot_fits_proj_scens(baserunname, scens, projfits, fitdict, bounds, 
                            fits_y_labels, basevar='RepVar', endyear=str(endyear))
fig1 = plot_fits_proj_scens(baserunname, scens, projfits, fitdict, bounds, 
                            fits_y_labels, endyear=str(endyear))
fig0.savefig(f"./{baserunname}_fits_proj_scens_noise.{figext}", dpi=figdpi, bbox_inches='tight')
fig1.savefig(f"./{baserunname}_fits_proj_scens.{figext}", dpi=figdpi, bbox_inches='tight')
# fig0.savefig(f"./{baserunname}_fits_proj_scens_noise.jpg", bbox_inches='tight')
# fig1.savefig(f"./{baserunname}_fits_proj_scens.jpg", bbox_inches='tight')

In [ ]:
##### PLOT RATE-STOCK-HAZARD GRAPHS #####

# Read in main and sensitivity results and truncate to just historical portion
t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
t = t.loc[:, :f'{endyear}']
s = pd.read_csv(f'{baserunname}_sens_{basescens[0][:-4]}_clean.tab', sep='\t', index_col=[0,1])
s = s.loc[:, :f'{float(endyear)}']

# Establish list of rates-stocks-hazards-titles-stocknames to plot
rateslist = [
    {'r': 'Initiating Rx misuse own Rx', 's': 'Patients with current month opioid Rx', 
     'h': 'Initiation rate Rx misuse own Rx effective', 
     't': 'A) Initiating Rx misuse \nfrom medical use', 'n': 'Rx opioid medical use'}, 
    {'r': 'Initiating heroin with Rx misuse', 's': 'Rx misuse no PY heroin', 
     'h': 'Initiation rate heroin with Rx misuse effective', 
     't': 'B) Initiating heroin \nfrom Rx misuse', 'n': 'Rx misuse'}, 
    {'r': 'Initiating heroin with Rx OUD', 's': 'Rx OUD no PY heroin no MOUD', 
     'h': 'Initiation rate heroin with Rx OUD effective', 
     't': 'C) Initiating heroin \nfrom Rx OUD', 'n': 'Rx OUD \n(no H use)'}, 
#     {'r': 'Developing HUD with Rx OUD', 's': 'Rx OUD with PY heroin no MOUD', 
#      'h': 'Developing HUD rate with Rx OUD effective', 't': 'D) Developing HUD \nfrom Rx OUD', 
#      'n': 'Rx OUD (w/ H use)'}, 
    {'r': 'Overdose death Rx OUD no H', 's': 'Rx OUD no PY heroin no MOUD', 
     'h': 'Overdose death rate Rx OUD no H', 't': 'D) Overdose death \nfrom Rx OUD (no H use)', 
     'n': 'Rx OUD \n(no H use)'}, 
    {'r': 'Overdose death HUD', 's': 'HUD no MOUD', 'h': 'Overdose death rate HUD', 
     't': 'E) Overdose death \nfrom HUD', 'n': 'HUD'}
]

# Define y-axis labels
ylabels = ['Transition \n(thousand people/year)', 
           'Transition hazard \nrate, normalised', 
           'Source population, \nnormalised']

# Set up main figure with three rows of axes
fig0, axs = plt.subplots(3, 5, figsize=[7, 4], sharex=True, constrained_layout=True)

# Set up sub-lists of axes for rows and columns
row0, row1, row2 = axs
col_list = [col for col in zip(row0, row1, row2)]

# Set up y-axis sharing 
for row in axs:
    for ax in row[1:3]:
        ax.sharey(row[0])
        ax.tick_params(labelleft=False)
    
    row[3].sharey(row[4])
    row[4].tick_params(labelleft=False)

# Iterate through columns/rates and plot lines and credible regions on each 
for rate, col in zip(rateslist, col_list):
    axa, axb, axc = col # Unpack column to three separate axes
    
    line0, = axa.plot(t.loc[rate['r']])
    line1, = axb.plot(t.loc[rate['h']]/t.loc[rate['h'], f'{styear}'], color='r')
    line2, = axc.plot(t.loc[rate['s']]/t.loc[rate['s'], f'{styear}'], color='g')
    for b in bounds:
        axa.fill_between(t.columns, s.loc[rate['r'], b[0]], 
                         s.loc[rate['r'], b[1]], alpha=0.2)
        axb.fill_between(t.columns, s.loc[rate['h'], b[0]]/t.loc[rate['h'], f'{styear}'], 
                         s.loc[rate['h'], b[1]]/t.loc[rate['h'], f'{styear}'], 
                         alpha=0.2, color='r')
        axc.fill_between(t.columns, s.loc[rate['s'], b[0]]/t.loc[rate['s'], f'{styear}'], 
                         s.loc[rate['s'], b[1]]/t.loc[rate['s'], f'{styear}'], 
                         alpha=0.2, color='g')

    # Add stocknames from rateslist as labels to second row
    axc.text(0.03, 0.97, rate['n'], transform=axc.transAxes, color='gray', va='top', fontsize=6)
    
    # Set up axis labels & title
    loc = mticker.MultipleLocator(base=8)
    axc.xaxis.set_major_locator(loc)
    axc.tick_params(axis='x', labelsize=6, labelrotation=90)
    for ax in col:
        ax.tick_params(axis='y', labelsize=6)
    axa.set_title(rate['t'], fontsize=8) # Take title from rateslist

# Format y-axis and set labels
for ax in row0:
    ax.yaxis.set_major_formatter(lambda x, pos: f'{x / 1e3:.0f}')

for row, label in zip(axs, ylabels):
    row[0].set_ylabel(label, fontsize=6) # Put label only on leftmost axes

fig0.suptitle('Changes in state transitions (flows), hazard rates, '
              + 'and source populations (states) over time', fontsize=9)

# Set up and display legend
handles = [line0, line1, line2]
labels = ['Transition (people/year)', 
          'Source population (people) relative to initial value', 
          'Transition hazard rate (per person-year) relative to initial value']
fig0.legend(handles, labels, loc=9, bbox_to_anchor=(0.5, 0), ncol=2, fontsize=6)

fig0.savefig(f"./{baserunname}_rates_3row_noUD.{figext}", dpi=figdpi, bbox_inches='tight')


In [ ]:
##### PLOT FENTANYL AND NALOXONE COUNTERFACTUALS GRAPHS #####

# Read in base and no-fentanyl scenario results and truncate to just historical portion
table = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
nft = pd.read_csv(f'{baserunname}_final_NoFentHist_vars.tab', sep='\t', index_col=0)
table = table.loc[:, :f'{endyear}']
nft = nft.loc[:, :f'{endyear}']

# Compile separate dataframe of just relevant results
baseod = table.loc['Total overdose deaths']
nxodav = (table.loc['Nx utilization deaths averted Rx user'] 
          + table.loc['Nx utilization deaths averted H user'])
nofent = nft.loc['Total overdose deaths']
nfdata = (table.loc['Total overdose deaths base Rx NVSS'] 
          + table.loc['Total overdose deaths base heroin NVSS'])
nfnxav = (nft.loc['Nx utilization deaths averted Rx user'] 
          + nft.loc['Nx utilization deaths averted H user'])
oddata = table.loc['Total overdose deaths NVSS']

odcomp = pd.concat([baseod, nxodav, nofent, nfdata, nfnxav, oddata], 
                   keys=['Base', 'NxDA', 'NoFent', 'NFData', 'NFNxDA', 'ODData'], axis=1)

del table, nft # Clear large dataframes to free up memory

# Calculate sums to stack graphs
odcomp['NoNx'] = odcomp['Base'] + odcomp['NxDA']
odcomp['NFNoNx'] = odcomp['NoFent'] + odcomp['NFNxDA']

# Interpolate data to allow complete line plotting
odcomp['NFData'] = odcomp['NFData'].interpolate()
odcomp['ODData'] = odcomp['ODData'].interpolate()

# Set up two-axes figure
fig0, ax0 = plt.subplots(figsize=[3.5, 3], constrained_layout=True)

# Plot naloxone results on first axes
based = ax0.plot(odcomp['ODData'], ls='dashed', color='b', label='Baseline (data)')
base = ax0.plot(odcomp['Base'], color='b', zorder=10, label='Baseline (simulated)')
nonx = ax0.plot(odcomp['NoNx'], color='g', label='No layperson naloxone (simulated)')
nfntd = ax0.plot(odcomp['NFData'], ls='dashed', color='r', label='No synthetic opioids (data)')
nfnt = ax0.plot(odcomp['NoFent'], color='r', label='No IMF (simulated counterfactual)')

ax0.fill_between(odcomp.index, odcomp['Base'], odcomp['NoNx'], color='g', alpha=0.2)
ax0.fill_between(odcomp.index, odcomp['Base'], odcomp['NoFent'], color='r', alpha=0.2)
ax0.legend(fontsize=6)

loc = mticker.MultipleLocator(base=8)
ax0.xaxis.set_major_locator(loc)
ax0.tick_params(axis='x', labelsize=6, labelrotation=90)
ax0.tick_params(axis='y', labelsize=6)
    
# Set labels for left-hand axes only, and figure title
ax0.set_ylabel('Overdose deaths (people/year)', fontsize=8)
fig0.suptitle('Impacts of naloxone and IMF on opioid overdose deaths', fontsize=8)

fig0.savefig(f"./{baserunname}_fentnxodsingle.{figext}", dpi=figdpi, bbox_inches='tight')

In [ ]:
##### PLOT ALL VARS FIT TO DATA & PROJECTIONS #####

def plot_fits_full(table, fullfits, fitdict):
    """Plot fit-to-data for all variables specified in `fullfits` list, 
    with names from `fitdict`"""
    
    # Define main figure and axes
    fig0, axs = plt.subplots(4, 4, figsize=[14, 10], sharex=True, constrained_layout=True)

    # Plot sim output and data
    for var, ax in zip(fullfits, axs.flatten()):
        ax.plot(table.loc[f'SimVar[{var}]'])
        ax.scatter(table.columns[::4], table.loc[f'DataVar[{var}]'][::4], color='silver')
        ax.set_title(fitdict[var], fontsize=9)

        # Format X and Y axis for all axes
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=7, labelrotation=90)

        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3))
        ax.yaxis.offsetText.set_fontsize(7)
        ax.tick_params(axis='y', labelsize=7)
    
    return fig0


def plot_fits_proj_full(table, senstable, fullfits, fitdict, 
                        bounds, basevar='SimVar', noisevar=None, endyear='2019'):
    """Plot projections for all variables specified in `fullfits` list, 
    with CrIs specified in `bounds` (multiple CrIs allowed, use paired 
    lower/upper values) and names from `fitdict`; show CrIs without 
    noise by default; show CrIs with noise if `noisevar` specified 
    (usually as RepVar)"""
    t, s = table, senstable # For more compact code
    
    # Define main figure and axes
    fig0, axs = plt.subplots(4, 4, figsize=[14, 10], sharex=True, constrained_layout=True)
    
    # Plot sim output and CI bounds
    for ax, var in zip(axs.flatten(), fullfits):
        ax.plot(t.columns, t.loc[f'{basevar}[{var}]'])
        for bd in bounds:
            if noisevar: # Additionally show CIs for noise if specified
                ax.fill_between(t.columns, s.loc[f'{noisevar}[{var}]', bd[0]], 
                                s.loc[f'{noisevar}[{var}]', bd[1]], alpha=0.2, color='grey')
            ax.fill_between(t.columns, s.loc[f'{basevar}[{var}]', bd[0]], 
                            s.loc[f'{basevar}[{var}]', bd[1]], alpha=0.2)
        ax.set_title(fitdict[var], fontsize=9)
        
        # Add vertical dashed line to mark end of data
        ax.vlines(endyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
        
        # Format X and Y axis for all axes
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=7, labelrotation=90)

        ax.set_ylim(0) # Anchor y axis on 0
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3))
        ax.yaxis.offsetText.set_fontsize(7)
        ax.tick_params(axis='y', labelsize=7)

    return fig0


# Specify all vars to be included
fullfits = ['InRM', 'InRD', 'RMis', 'NDHU', 'ROUD', 'ROUH', 'HUD', 'TxBp', 
            'InHD', 'InHR', 'InHO', 'InHT', 'ODRB', 'ODSB', 'ODHC', 'ODTo']

# Read in and plot fits to data
table = pd.read_csv(f'{baserunname}_main_fits.tab', sep='\t', index_col=0)
fig0 = plot_fits_full(table, fullfits, fitdict)
fig0.savefig(f"./{baserunname}_fits_full.{figext}", dpi=figdpi, bbox_inches='tight')

# Read in main and sensitivity results to plot all vars projections for each base scenario
for scen in (basescens + scenariolist):
    t = pd.read_csv(f'{baserunname}_final_{scen[:-4]}_vars.tab', sep='\t', index_col=0)
    s = pd.read_csv(f'{baserunname}_sens_{scen[:-4]}_clean.tab', sep='\t', index_col=[0,1])
    fig1 = plot_fits_proj_full(t, s, fullfits, fitdict, bounds, endyear=str(endyear))
    fig1.savefig(f"./{baserunname}_fits_proj_full_{scen[:-4]}.{figext}", dpi=figdpi, bbox_inches='tight')


In [ ]:
##### PLOT HOLDOUT DATA ALL VARS PROJECTIONS #####

def plot_fits_proj_hold(table, senstable, datatable, fullfits, fitdict, 
                        bounds, basevar='SimVar', noisevar=None, endyear='2020', holdoutyear='2015'):
    """Plot projections for all variables specified in `fullfits` list, 
    with CrIs specified in `bounds` (multiple CrIs allowed, use paired 
    lower/upper values) and names from `fitdict`; show CrIs without 
    noise by default; show CrIs with noise if `noisevar` specified 
    (usually as RepVar)"""
    t, s, d = (table.loc[:, :endyear], senstable.loc[:, :str(float(endyear))], datatable) # For more compact code
    
    # Define main figure and axes
    fig0, axs = plt.subplots(4, 4, figsize=[14, 10], sharex=True, constrained_layout=True)
    
    # Plot sim output and CI bounds
    for ax, var in zip(axs.flatten(), fullfits):
        ax.plot(t.columns, t.loc[f'{basevar}[{var}]'])
        ax.scatter(t.columns[::4], d.loc[f'DataVar[{var}]'][::4], color='silver')
        for bd in bounds:
            if noisevar: # Additionally show CIs for noise if specified
                ax.fill_between(t.columns, s.loc[f'{noisevar}[{var}]', bd[0]], 
                                s.loc[f'{noisevar}[{var}]', bd[1]], alpha=0.2, color='grey')
            ax.fill_between(t.columns[::4], s.loc[f'{basevar}[{var}]', bd[0]][::4], 
                            s.loc[f'{basevar}[{var}]', bd[1]][::4], alpha=0.2)
        ax.set_title(fitdict[var], fontsize=9)
        
        # Add vertical dashed line to mark end of data
        ax.vlines(holdoutyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
        
        # Format X and Y axis for all axes
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=7, labelrotation=90)

        ax.set_ylim(0) # Anchor y axis on 0
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3))
        ax.yaxis.offsetText.set_fontsize(7)
        ax.tick_params(axis='y', labelsize=7)

    return fig0


t = pd.read_csv(f'{baserunname}_hold_final_Base_vars.tab', sep='\t', index_col=0)
s = pd.read_csv(f'{baserunname}_hold_sens_Base_clean.tab', sep='\t', index_col=[0,1])
d = pd.read_csv(f'{baserunname}_main_fits.tab', sep='\t', index_col=0)
fig0 = plot_fits_proj_hold(t, s, d, fullfits, fitdict, bounds, basevar='RepVar', endyear=str(endyear), holdoutyear=str(holdoutyear))
fig0.savefig(f"./{baserunname}_hold_fits_full_1A.{figext}", dpi=figdpi, bbox_inches='tight')


In [ ]:
##### SYNDATA INTERVALS GRAPH AND PARAM ESTIMATES MEGA-PLOT #####

def plot_syndata_intervals(means, percs):
    """Plot percent of syndata estimates within each theoretical CrI"""
    spmdf = means.iloc[0:len(percs)] * 100 # Select relevant rows of means df
    spmdf.index = spmdf.index.astype('int')
    spmdf.loc[0] = 0 # Anchor results on (0,0)
    spmdf.sort_index(inplace=True)
    
    # Create figure, then plot and label results
    fig0, ax0 = plt.subplots(figsize=[7,6.5], constrained_layout=True)

    ax0.plot(spmdf)
    ax0.scatter(spmdf.index, spmdf.values, marker='s')
    ax0.plot([0,100], [0,100], ls='dashed') # Plot 1:1 line

    ax0.set_xlabel('Theoretical credible interval (percent)')
    ax0.set_ylabel('Estimated parameters inside CI (percent)')
    fig0.suptitle('Synthetic data estimation credible interval accuracy', fontsize=14)
    
    return fig0


def plot_syndata_params(syndf, synpercdf, perc):
    """Plot parameter estimates, CIs, and true values for syndata output 
    for all params and syndata runs in single mega-plot"""
    varnum = len(syndf.columns) # Get number of vars to plot

    # Create figure laying out axes in rows of three
    fig0, axs = plt.subplots(int(np.ceil(varnum/3)), 3, 
                             figsize=[12, np.ceil(varnum/3) * 1.5], constrained_layout=True)

    # Delete any excess axes
    for i in range(len(axs.flatten())-varnum):
        fig0.delaxes(axs.flatten()[varnum + i])

    # Iterate over axes & vars to plot each one
    for ax, var in zip(axs.flatten(), syndf.columns):
        s = syndf[var]

        bds = strbds_from_perc(perc) # Get bounds for errorbars from specified interval
        errs = ((s['0.5'] - s[bds[0]]), s[bds[1]] - s['0.5'])

        # Plot estimated value with CI and true value
        ax.errorbar(s['Value'].index, s['Value'].values, yerr=errs, fmt='o', capsize=3)
        ax.scatter(s['True'].index, s['True'].values, marker='x', color='r', s=9**2)
        
        # Set title including % values within bounds
        ax.set_title(f'{var} ({synpercdf[var].loc[str(perc)].mean()})', fontsize=9)

        # Format x and y axis
        loc = mticker.MultipleLocator(base=1)
        ax.xaxis.set_major_locator(loc)
        ax.xaxis.set_ticklabels([])

        ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
        ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
        ax.yaxis.offsetText.set_fontsize(7)
        ax.tick_params(axis='y', labelsize=7)
    
    return fig0


# Plot theoretical CrI vs. actual included values comparison
means = pd.read_csv(f'{baserunname}_syndata_means.tab', sep='\t', index_col=0)
fig0 = plot_syndata_intervals(means, syn_reppercs)
fig0.savefig(f"./{baserunname}_syn_intervals.{figext}", dpi=figdpi, bbox_inches='tight')

# Plot syndata param estimates and true values, breaking up mega-plot to fit on separate pages
syndf = pd.read_csv(f'{baserunname}_syndata_results.tab', sep='\t', index_col=[0, 1])
syndf = syndf.reorder_levels(['Perc', 'Run']).sort_index()
synpercdf = pd.read_csv(f'{baserunname}_syndata_intervals.tab', sep='\t', index_col=[0, 1])

fig0 = plot_syndata_params(syndf.iloc[:, 0:33], synpercdf.iloc[:, 0:33], syn_mainperc)
fig0.savefig(f'./{baserunname}_syn_params_0.{figext}', dpi=figdpi, bbox_inches='tight')
fig1 = plot_syndata_params(syndf.iloc[:, 33:66], synpercdf.iloc[:, 33:66], syn_mainperc)
fig1.savefig(f'./{baserunname}_syn_params_1.{figext}', dpi=figdpi, bbox_inches='tight')
fig2 = plot_syndata_params(syndf.iloc[:, 66:], synpercdf.iloc[:, 66:], syn_mainperc)
fig2.savefig(f'./{baserunname}_syn_params_2.{figext}', dpi=figdpi, bbox_inches='tight')

In [ ]:
##### LOOP KNOCKOUT ANALYSIS GRAPHS #####

# Set up labels for loop knockout keys
lknamedict = {'av': 'Availability', 'pr': 'Perceived risk', 'si': 'Social influence'}

# Iterate through loop knockout keys and plot results from each
for key in lkdict.keys():
    for name in ['lk_run', 'lk']:
        t = pd.read_csv(f'{baserunname}_{name}_{key}_{basescens[0][:-4]}_vars.tab', 
                        sep='\t', index_col=0)
        # Restrict to historical portion of run
        t = t.loc[:, :f'{endyear}']

        # Plot results for visual analysis
        fig0 = plot_fits_full(t, fullfits, fitdict)
        fig0.suptitle(f"Fits to data for run {name}_{key}", fontsize=14)
        fig0.savefig(f"./{baserunname}_{name}_{key}_fits.{figext}", dpi=figdpi, bbox_inches='tight')


In [ ]:
##### ANALYTICAL FUNCTION GRAPHING SHOWING PARAMETER SENSITIVITY #####

def prob_nx(x, k):
    return 1 - np.exp(-k * x)

def bup_cap_avg(x, k, b):
    return (b * np.exp(-k * x) / -k + b/k)/x

def bup_cap_marginal(x, k, b):
    return b * np.exp(-k * x)

def tx_success(x, i, k, l, m):
    y = np.where(x < i, 
                 k ** 2 / (1 + k ** 2) * np.exp((l / k * (x - i))), 
                 1 - np.exp(-l * k * (x - i)) / (1 + k ** 2)) * m
    return y


def eval_func(func, xrng, args, sensrange=0.2):
    """Umbrella function for calculating results of `func` with `args` 
    over range `xrng`, varying each arg over +/- `sensrange` values"""
    x = np.linspace(*xrng, 100)
    y = []
    for i in range(len(args)):
        args_l = args.copy()
        args_l[i] = args[i] * (1 - sensrange)
        args_h = args.copy()
        args_h[i] = args[i] * (1 + sensrange)
        y.append((func(x, *args), func(x, *args_l), func(x, *args_h)))
    return x, y

def plot_func(x, y, axs):
    """Plot function output from `eval_func`, with +/- arg values"""
    for (m, l, h), ax in zip(y, axs):
        ax.plot(x, m, c='k', lw=3, zorder=10, label='Base')
        ax.plot(x, l, label='Low')
        ax.fill_between(x, m, l, alpha=0.2)
        ax.plot(x, h, label='High')
        ax.fill_between(x, m, h, alpha=0.2)


# Set mdl and out filenames from which to read parameter values
mdl = f"../{simsettings['model']}"
out = f'../{baserunname}_main_full.out'

# Evaluate and plot Nx distribution efficiency function
k = get_value(out, 'Nx kit distribution efficiency')
x, y = eval_func(prob_nx, [0, 6000], [k])
fig0, ax0 = plt.subplots(figsize=[7, 5], constrained_layout=True)
axs = [ax0] # plot_func reads in axs as list
plot_func(x, y, axs)
ax0.set_xlabel(r'Layperson naloxone kits distributed per 100K population, $\nu_{(.)}$')
ax0.set_ylabel('Probability of bystander naloxone administration \n' 
               + r'for witnessed overdose, $p_{IB(.)}$')
ax0.set_title(r'Effect of naloxone kit distribution efficiency, $\lambda_N$', fontsize=14)
ax0.legend(loc=4)
fig0.savefig(f"./Function_ProbNx.{figext}", dpi=figdpi, bbox_inches='tight')

# Evaluate and plot marginal bup cap per provider function
k = get_value(mdl, 'Bup effective capacity decay constant')
b = get_value(mdl, 'Bup effective capacity per provider base')
x, y = eval_func(bup_cap_marginal, [0, 120000], [k, b])
fig0, (ax0, ax1) = plt.subplots(1, 2, figsize=[10, 5], sharey=True, constrained_layout=True)
axs = [ax0, ax1]
plot_func(x, y, axs)
fig0.supxlabel(r'Buprenorphine waivered providers, $B$')
fig0.suptitle(r'Effect of parameters on marginal effective capacity per new Bup provider, '
              r'$\widehatK$', fontsize=14)
ax0.set_ylabel(r'Marginal effective capacity per new Bup provider, $\widehatK$')
ax0.set_title(r'Bup effective capacity decay constant, $\lambda_B$')
ax1.set_title(r'Bup effective capacity per provider base, $\widehatK_0$')
ax0.legend()
ax1.legend()
fig0.savefig(f"./Function_BupCap.{figext}", dpi=figdpi, bbox_inches='tight')

# Evaluate and plot Tx duration-success function
i, k, l, m = [get_value(mdl, f'Tx success fraction {var}') 
              for var in ['inflection', 'kappa', 'lambda', 'max']]
x, y = eval_func(tx_success, [0, 5], [i, k, l, m])
fig0, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[10, 8], sharex=True, sharey=True, 
                                              constrained_layout=True)
axs = [ax0, ax1, ax2, ax3]
plot_func(x, y, axs)
fig0.supxlabel(r'Average duration of treatment, $\tau_{T(.)}$ (years)')
fig0.supylabel(r'Treatment success fraction, $p^R_{(.)}$')
fig0.suptitle(r'Effect of parameters on treatment success fraction, $p^R_{(.)}$', fontsize=14)
ax0.set_title(r'Inflection point, $m_R$')
ax1.set_title(r'Asymmetry parameter kappa, $\kappa_R$')
ax2.set_title(r'Scale parameter lambda, $\lambda_R$')
ax3.set_title(r'Max success fraction, $p^{RM}$')
for ax in axs:
    ax.legend()
fig0.savefig(f"./Function_TxSF.{figext}", dpi=figdpi, bbox_inches='tight')

# Evaluate and plot bup patients per provider function and data
# Read in and subset necessary data
t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
t = t.loc[:, :f'{endyear}']
prov = t.loc['Bup providers DATA']
pats = t.loc['Tx point patients Bup DATA']
pppr = t.loc['Bup patients per provider DATA']
pppn = t.loc['Bup effective capacity per provider net']

# Evaluate and plot average bup cap function and patients per provider data
k = get_value(mdl, 'Bup effective capacity decay constant')
b = get_value(mdl, 'Bup effective capacity per provider base')
x, y = eval_func(bup_cap_avg, [0, 75000], [k, b])
fig0, ax0 = plt.subplots(figsize=[8,6], constrained_layout=True)
ax0.plot(x, y[0][0], label='Avg. effective capacity per provider (indicated)')

# Add line for net capacity per provider
ax0.plot(prov.interpolate(), pppn, label='Avg. effective capacity per provider (net)')

# Add and label historical patients per provider datapoints
ax0.scatter(prov, pppr, marker='x', color='r', label='Patients per provider (data)')
for i in t.columns[12:]:
    ax0.annotate(i, (prov[i], pppr[i]), xytext=(-10, 5), textcoords='offset points', fontsize=8)

ax0.set_xlabel(r'Buprenorphine waivered providers, $B$')
ax0.set_ylim(ymax=37) # Limit Y axis to reduce compactness of graph
ax0.set_ylabel(r'Patients per provider, $K_B/B$')
fig0.suptitle(r'Calculated vs. observed avg. patients or capacity per provider, $K_B/B$', 
              fontsize=14)
ax0.legend()
del t
fig0.savefig(f"./Function_BupData.{figext}", dpi=figdpi, bbox_inches='tight')


In [ ]:
##### HORIZONTAL FIT-TO-DATA & PROJECTIONS PLOTS #####

def plot_fits_main_horizontal(fits, mainfits, fitdict, ylabels, errmult=1.96):
    """Plot horizontal version of `plot_fits_main` with minor graphical 
    changes, for use in presentations"""
    
    # Define main figure and axes
    fig0, ((ax0, ax1, ax2), (ax3, ax4, ax5)) = plt.subplots(
        2, 3, figsize=[16, 7], sharex=True, constrained_layout=True)
    ax_list = [ax0, ax3, ax1, ax4, ax2, ax5]

    for ax, var in zip(ax_list, mainfits):
        ax.plot(fits.columns, fits.loc[f'SimVar[{var}]'], lw=3, 
                color='#007CBA', zorder=10, label='Simulated') # Use FDA blue for lines
        ax.scatter(fits.columns[::4], fits.loc[f'DataVar[{var}]'][::4], 
                   color='dimgrey', label='Historical')
        try:
            ax.errorbar(fits.columns[::4], fits.loc[f'DataVar[{var}]'][::4], 
                        yerr=fits.loc[f'DataErr[{var}]'][::4] * errmult, fmt='o', 
                        color='dimgrey', capsize=3, label='Historical')
        except KeyError: # Catch exception when DataErr not available
            pass
        
        # Format X and Y axis for all axes
        ax.set_ylim(0) # Anchor y axis on 0
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
        ax.ticklabel_format(axis='y', style='sci', scilimits=(6,6))
        
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=8)
        
    # Set labels for top row in lower left, bottom row in upper left
    for ax, label, var in zip([ax0, ax1, ax2], ylabels, mainfits[::2]):
        ax.set_ylabel(label)
        ax.text(0.05, 0.05, fitdict[var], transform=ax.transAxes, fontsize=14, va='bottom')
    
    for ax, label, var in zip([ax3, ax4, ax5], ylabels, mainfits[1::2]):
        ax.set_ylabel(label)
        ax.text(0.05, 0.95, fitdict[var], transform=ax.transAxes, fontsize=14, va='top')
        
    for ax in [ax2, ax5]:
        # Format Y-axis ticks without scientific notation
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
        ax.ticklabel_format(axis='y', style='sci', scilimits=(3,3))
        
    fig0.savefig(f"./{baserunname}_fits_main_horizontal.{figext}", dpi=figdpi, bbox_inches='tight')


def plot_fits_proj_horizontal(table, senstable, projfits, fitdict, bounds, 
                              ylabels, basevar='SimVar', endyear='2019'):
    """Plot horizontal version of `plot_fits_proj_scens` with single 
    scenario only, for use in presentations"""
    t, s = table, senstable # For more compact code
    
    # Define main figure and axes
    fig0, ((ax0, ax1, ax2), (ax3, ax4, ax5)) = plt.subplots(
        2, 3, figsize=[16, 7], sharex=True, constrained_layout=True)
    ax_list = [ax0, ax3, ax1, ax4, ax2, ax5]

    # Plot sim output and CrI bounds
    for ax, var in zip(ax_list, projfits):
        ax.plot(t.columns, t.loc[f'{basevar}[{var}]'], color='#007CBA') # Use FDA blue for lines

        for bd in bounds:
            ax.fill_between(t.columns, s.loc[f'{basevar}[{var}]', bd[0]], 
                            s.loc[f'{basevar}[{var}]', bd[1]], color='#007CBA', alpha=0.2)
        
        # Add vertical dashed line to mark end of data
        ax.vlines(endyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
        
        # Format X and Y axis for all axes
        ax.set_ylim(0) # Anchor y axis on 0
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
        ax.ticklabel_format(axis='y', style='sci', scilimits=(6,6))
        
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=8, labelrotation=90)

    # Set labels for top row in lower left, bottom row in upper left
    for ax, label, var in zip([ax0, ax1, ax2], ylabels, projfits[::2]):
        ax.set_ylabel(label)
        ax.text(0.05, 0.05, fitdict[var], transform=ax.transAxes, fontsize=14, va='bottom')
    
    for ax, label, var in zip([ax3, ax4, ax5], ylabels, projfits[1::2]):
        ax.set_ylabel(label)
        ax.text(0.05, 0.95, fitdict[var], transform=ax.transAxes, fontsize=14, va='top')
        
    for ax in [ax2, ax5]:
        # Format Y-axis ticks without scientific notation
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
        ax.ticklabel_format(axis='y', style='sci', scilimits=(3,3))
    
    fig0.savefig(f"./{baserunname}_fits_proj_horizontal.{figext}", dpi=figdpi, bbox_inches='tight')


# Add display labels to fitdict
fitdict['ODRT'] = 'Overdose deaths \n(Rx & Rx synthetics)'    
fitdict['ODHC'] = 'Overdose deaths (heroin \n& illicit synthetics)'

# Read in data to plot and subset as necessary
fits = pd.read_csv(f'{baserunname}_main_fits.tab', sep='\t', index_col=0)
fits = fits.loc[:, :f'{endyear}']
t = pd.read_csv(f'{baserunname}_final_{scenariolist[1][:-4]}_vars.tab', sep='\t', index_col=0)
s = pd.read_csv(f'{baserunname}_sens_{scenariolist[1][:-4]}_clean.tab', sep='\t', index_col=[0,1])

# Create plots with grayed out axes for better visual contrast
with plt.rc_context({'axes.edgecolor':'gray', 'xtick.color':'gray', 
                     'ytick.color':'gray', 'axes.labelcolor':'gray'}):
    plot_fits_main_horizontal(fits, mainfits, fitdict, fits_y_labels)
    plot_fits_proj_horizontal(t, s, projfits, fitdict, bounds, fits_y_labels, endyear=str(endyear))

In [ ]:
##### TWO-SCENARIO FULL PROJECTIONS PLOT #####

def plot_scens_proj_full(baserunname, scens, fullfits, fitdict, 
                         bounds, basevar='SimVar', noisevar=None, endyear='2020'):
    """Plot projections for all variables specified in `fullfits` list, 
    with CrIs specified in `bounds` (multiple CrIs allowed, use paired 
    lower/upper values) and names from `fitdict`; show CrIs without 
    noise by default; show CrIs with noise if `noisevar` specified 
    (usually as RepVar)"""

    # Define main figure and axes
    fig0, axs = plt.subplots(4, 4, figsize=[14, 10], sharex=True, constrained_layout=True)
    
    # Plot sim output and CI bounds for each scenario
    for ax, var in zip(axs.flatten(), fullfits):
        for i, scen in enumerate(scens):
            # Read in main and sensitivity output files
            t = pd.read_csv(f'{baserunname}_final_{scen[:-4]}_vars.tab', sep='\t', index_col=0)
            s = pd.read_csv(f'{baserunname}_sens_{scen[:-4]}_clean.tab', sep='\t', index_col=[0,1])

            if i > 0: # Subset to just projection portion for all but first scenario
                t = t.loc[:, endyear:]
                s = s.loc[:, endyear:]
                
            # Plot lines with each consecutive scenario lower in zorder
            ax.plot(t.columns, t.loc[f'{basevar}[{var}]'], zorder=(3-i/10))
            for bd in bounds:
                ax.fill_between(t.columns[::4], s.loc[f'{basevar}[{var}]', bd[0]][::4], 
                                s.loc[f'{basevar}[{var}]', bd[1]][::4], alpha=0.2, zorder=(3-i/10))
        
        ax.set_title(fitdict[var]) # Set axes titles based on fitdict
        
        # Add vertical dashed line to mark end of data
        ax.vlines(endyear, 0, 1, transform=ax.get_xaxis_transform(), color='r', ls='dashed')
    
        # Format X and Y axis for all axes
        loc = mticker.MultipleLocator(base=8)
        ax.xaxis.set_major_locator(loc)
        ax.tick_params(axis='x', labelsize=7, labelrotation=90)

        ax.set_ylim(0) # Anchor y axis on 0
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3))
        ax.yaxis.offsetText.set_fontsize(7)
        ax.tick_params(axis='y', labelsize=7)

    # Create figure legend with proxy artists for labels
    FentH = mlines.Line2D([], [], label='High Fentanyl', color='tab:blue')
    FentL = mlines.Line2D([], [], label='Low Fentanyl', color='tab:orange')
    handles = [FentH, FentL]
    fig0.legend(handles=handles, loc=9, bbox_to_anchor=(0.5, 0), ncol=len(handles))
        
    return fig0


# Specify all vars to be included
fullfits = ['InRM', 'InRD', 'RMis', 'NDHU', 'ROUD', 'ROUH', 'HUD', 'TxBp', 
            'InHD', 'InHR', 'InHO', 'InHT', 'ODRB', 'ODSB', 'ODHC', 'ODTo']

fig0 = plot_scens_proj_full(baserunname, basescens[-2:], fullfits, fitdict, bounds, endyear=str(endyear))
fig0.savefig(f"./{baserunname}_fits_proj_full_FentLH.{figext}", dpi=figdpi, bbox_inches='tight')

In [ ]:
##### SEND FIGURES TO SUBFOLDER FOR EASY ACCESS #####

os.chdir(basedir)
os.chdir('./Results')
os.makedirs('./Figures', exist_ok=True)

# for path, dnames, fnames in os.walk('./'):
#     for f in fnames:
#         if f.endswith(f'.{figext}'):
#             copy(f, './Figures')

for f in os.listdir():
    if f.endswith(f'.{figext}'):
        copy(f, './Figures')
        print(f)

In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000